# Play MountainCar

In [ ]:
import gym
env = gym.make("MountainCar-v0")

In [ ]:
import gym
env = gym.make('MountainCar-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        action = 2
        #action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

# Explore game environment

In [ ]:
import numpy as np
action_size = env.action_space.n #2: left / 1: right / 0:??
state = env.observation_space
print(action_size, state.shape)
print(state.low, state.high)

# Divide the environment into micro squares

In [ ]:
import math

state = env.observation_space
x_step = (state.high[0] - state.low[0])/100
y_step = (state.high[1] - state.low[1])/100
print("(x_step, y_step) = ", x_step, y_step)
x_range = [state.low[0] + j * x_step for j in range(100)]
y_range = [state.low[1] + j * y_step for j in range(100)]
def find_square(x, y):
    state = env.observation_space
    if x > state.high[0] or x < state.low[0]:
        raise IndexError
    if y > state.high[1] or y < state.low[1]:
        raise IndexError
    i = len(x_range) - 1
    while (x < x_range[i] and i > 0):
        i -= 1
    j = len(y_range) - 1
    while (y < y_range[j] and j > 0):
        j -= 1
    return (i, j)

speed_range = 10
def find_speed(speed):
    if speed < 0 or speed > speed_range:
        raise IndexError(speed)
    return math.ceil(speed/speed_range * speed_range)

qtable = np.zeros((len(x_range) , len(y_range), speed_range, action_size))
qtable.shape

# Train the model

In [ ]:
import random
import math
# List of rewards
rewards = []

total_episodes = 6000       # Total episodes
learning_rate = 0.7        # Learning rate
max_steps = 999             # Max steps per episode
gamma = 0.9                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    state = find_square(state[0], state[1])
    grad = [0 , 0]
    speed_state = 0
    max_x = min_x = state[0]
    max_y = min_y = state[1]
    step = 0
    done = False
    total_rewards = 0
    max_right = state[0]
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state[0], state[1], speed_state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
        
        if action > 2:
            print("c'est bizarre", action)
        # Take the action (a) and observe the outcome state(s') and reward (r)
        
        new_state, reward, done, info = env.step(action)
        new_state = find_square(new_state[0], new_state[1])
        
        grad[0] += (new_state[0] - state[0])
        grad[1] += (new_state[1] - state[1])
        speed = math.pow(grad[0], 2) + math.pow(grad[1], 2)
        speed = min(speed, speed_range - 1)
        speed_state = find_speed(speed)
        total_rewards += reward
        if reward == -1:
            if new_state[0] > max_right:
                reward = 5 * speed_state
                max_right = new_state[0]
            else:
                reward = speed_state
        
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state[0], state[1], speed_state, action] += learning_rate * (reward + gamma * np.max(qtable[new_state[0], new_state[1], speed_state, :]) - qtable[state[0], state[1], speed_state, action])
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True:
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(np.min(qtable))

# Play the game

In [ ]:
env.reset()

for episode in range(5):
    state = env.reset()
    state = find_square(state[0], state[1])
    grad = [0 , 0]
    #state = (90, 72)
    step = 0
    speed_state = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state[0], state[1], speed_state,:])
        #action = 2
        
        new_state, reward, done, info = env.step(action)
        new_state = find_square(new_state[0], new_state[1])
        
        grad[0] += (new_state[0] - state[0])
        grad[1] += (new_state[1] - state[1])
        speed = math.pow(grad[0], 2) + math.pow(grad[1], 2)
        speed = min(speed, speed_range - 1)
        speed_state = find_speed(speed)
        env.render()
        if done:
            env.render()
            print(new_state)
            print("reward:", reward)
            print("Number of steps", step)
            
            break
        state = new_state
env.close()